In [1]:
# defines functions that take in a STC number range and divide it up into the numbers between them
# defines a function that works out which of these functions to apply
# runs it on a copy of Pantzer dict and replaces the spans with all the numbers (taking the attributes from the original number)

import re

In [2]:
%store -r pantzer

In [3]:
# functions
def function1(string): # 1. integer to integer
    pair = string.split('—')
    newpair = [pair[0]]
    newpair.append(pair[0][:-len(pair[1])] + pair[1])
    
    span = [i for i in range(int(newpair[0]), int(newpair[1]) + 1)]
    return span

def function2(string): # 2. decimal to decimal, same integer (with or without int repeated)
    pair = string.split('—')
    leftsplit = pair[0].split('.')
    rightsplit = pair[1].split('.')
    root = leftsplit[0]
    lower = pair[0]
    upper = root + '.' + rightsplit[1]
    newpair = [lower, upper]
    
    span = []
    for i in range(int(leftsplit[1]), int(rightsplit[1]) + 1):
        span.append(root + '.' + str(i))
    return span

def function3(string): # 3. integer to decimal in same integer (with or without int repeated)
    pair = string.split('—')
    rightsplit = pair[1].split('.')
    lower = pair[0]
    root = lower
    newpair = [lower, root + '.' + rightsplit[1]]
    
    span = [lower]
    for i in range(1, int(rightsplit[1]) + 1):
        span.append(root + '.' + str(i))
    return span

def function4(string): # 4. decimal to integer
    pair = string.split('—')
    leftsplit = pair[0].split('.')
    root = leftsplit[0]
    decimal = leftsplit[1]
    upper = root[0:-len(pair[1])] + pair[1]
    newpair = [pair[0], upper]

    span = []
    for i in range(int(leftsplit[1]), 11): #going up to 10 here but i guess could be higher! 
        span.append(root + '.' + str(i))
    span.append(upper)
    return span

def function5(string): # decimal to decimal in another integer. only works if roots are adjacent!
    pair = string.split('—')
    leftsplit = pair[0].split('.')
    rightsplit = pair[1].split('.')
    lower = pair[0]
    upper = leftsplit[0][:-len(rightsplit[0])] + pair[1]
    newpair = [lower, upper]
    
    span = []
    for i in range(int(leftsplit[1]), 11):
        span.append(leftsplit[0] + '.' + str(i))
    for i in range(int(leftsplit[0]) + 1, int(upper.split('.')[0])):
        for d in range(0, 11):
            newnumber = str(i) + '.' + str(d)
            newnumber = newnumber.replace('.0', '')
            span.append(newnumber)
    for i in range(0, int(rightsplit[1]) + 1):
        newnumber = leftsplit[0][:-len(rightsplit[0])] + rightsplit[0] + '.' + str(i)
        newnumber = newnumber.replace('.0', '')
        span.append(newnumber)
    return span

def function6(string): # 6 integer to decimal, different integer
    pair = string.split('—')
    rightsplit = pair[1].split('.')
    lower = pair[0]
    upper = lower[:-len(rightsplit[0])] + pair[1]
    newpair = [lower, upper]
    
    span = []
    for i in range(int(lower), int(upper.split('.')[0])):
        for d in range(0, 11):
            newnumber = str(i) + '.' + str(d)
            newnumber = newnumber.replace('.0', '')
            span.append(newnumber)
    for i in range(0, int(rightsplit[1]) + 1):
        newnumber = upper.split('.')[0] + '.' + str(i)
        newnumber = newnumber.replace('.0', '')
        span.append(newnumber)
    return span

def function7(string): # 7 works for spans with an a version, like 1234-1234a
    pair = string.split('—')
    span = [pair[0], pair[0] + 'a']
    return span

In [4]:
letters = []

def rangeconversion(string):
    
    # check if contains letters first:
    lowerstring = string.lower()
    if lowerstring.islower():
        pair = string.split('—')
        if pair[1] == str(pair[0][-1]) + 'a':
            return function7(string)
        else:
            letters.append(string)
    
    else:
        pair = string.split('—')
        left = pair[0]
        right = pair[1]


        if '.' in left:
            if '.' in right:

                leftsplit = left.split('.')
                rightsplit = right.split('.')

                if len(rightsplit[0]) > 0: #check whether the right has a number before the point

                    if leftsplit[0].endswith(rightsplit[0]):
                        return function2(string)
                        kind = 'decimal to decimal, same integer, integer repeated' #solved 2.
                    else:
                        return function5(string)
                        kind = 'decimal to decimal, different integer' #solved 5.

                else:
                    return function2(string)
                    kind = 'decimal to decimal, same integer, point only' #solved 2.

            else:
                return function4(string)
                kind = 'decimal to integer' #solved 4.

        else:
            if '.' in right:

                rightsplit = right.split('.')

                if len(rightsplit[0]) > 0: #check whether the right has a number before the point
                    if left.endswith(rightsplit[0]):
                        return function3(string)
                        kind = 'integer to decimal, same integer, integer repeated' #solved 3.
                    else:
                        return function6(string)
                        kind = 'integer to decimal, different integer'
                else:
                    return function3(string)
                    kind = 'integer to decimal, same integer, point only' #solved 3.
            else:
                return function1(string)
                kind = 'integer to integer' #solved 1.

#     print(kind)

#             

In [5]:
# make a copy of pantzer dict to insert the created numbers

import copy
pantzer_max = copy.deepcopy(pantzer)

In [6]:
len(letters)

0

In [7]:
# run functions on every stc number with a dash

for name in pantzer.keys():
    for year in pantzer[name].keys():
        if year != 'headnote':
            for number in pantzer[name][year].keys():
                if '—' in number:
                    range_values = rangeconversion(number)
                    
                    if range_values:
                        for i in range_values:
                            pantzer_max[name][year][i] = pantzer[name][year][number]
                    
                    del pantzer_max[name][year][number]

                        

In [8]:
# SEARCH ENGINE

target = '19937'

printed_for = []
printed_by = []
attributes = []
for name in pantzer_max.keys():
    for year in pantzer_max[name].keys():
        if year != 'headnote':
            for number in pantzer_max[name][year].keys():
                if str(number).lower() == target:
                    
                    
                    
                    if 'ff.' not in pantzer_max[name][year][number] and 'f.' in pantzer_max[name][year][number]:
                        printed_for.append(name)
                    else:
                        printed_by.append(name)
                    
                    if pantzer_max[name][year][number] not in attributes:
                        attributes.append(pantzer_max[name][year][number])
                    
#                     print(name)
#                     print(year)
#                     print(number)
#                     print(pantzer_max[name][year][number])
#                     print(' ')

print('Printed by ' + '; '.join(printed_by))
if len(printed_for) < 0:
    print('For ' + '; '.join(printed_for))
print('Attributes: ' + ''.join(attributes))

Printed by 
Attributes: 


In [21]:
%store pantzer_max

Stored 'pantzer_max' (dict)


In [17]:
# search if string is *in* stc number, i.e. not exact match

target = '4474'

printed_for = []
printed_by = []
attributes = []
for name in pantzer_max.keys():
    for year in pantzer_max[name].keys():
        if year != 'headnote':
            for number in pantzer_max[name][year].keys():
                if target in str(number).lower():
                    
#                     if 'f.' in pantzer_max[name][year][number]:
#                         printed_for.append(name)
#                     else:
#                         printed_by.append(name)
                    
#                     if pantzer_max[name][year][number] not in attributes:
#                         attributes.append(pantzer_max[name][year][number])
                    
                    print(name)
                    print(year)
                    print(number)
                    print(pantzer_max[name][year][number])
                    print(' ')


Denham, Henry.
1580
4474.71
ex off.; 
 
Denham, Henry.
1580
4474.115
ex off.; 
 
Kitson, Abraham.
1589
24474
sold (d); 
 
Legge (Legg), Cantrell.
1612
4474.33
typis; 
 
Thomas, Thomas 1.
1585
4474.1
ex off.; 
 
Thomas, Thomas 1.
1585
4474.79
ex off.; 
 
Windet, John.
1589
24474
(d); 
 
